In [ ]:
!pip install --upgrade transformers
!pip install accelerate

In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)

# Now you can try to load the model
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

2025-07-18 18:16:36.143162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752862596.319225     203 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752862596.366913     203 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

Device set to use cuda:0


## Sample prompt

In [4]:
B_INST, E_INST = "<start_of_turn>user\n", "<end_of_turn>\n"
B_SYS, E_SYS = "<start_of_turn>system\n", "<end_of_turn>\n"

system_prompt = """You are an expert AI/ML competition analyst. 
Your task is to classify machine learning competitions into precise categories. 
Your answer MUST use standardized academic or industry terms. 
If information is unclear, answer with 'Unknown'. 
Do not guess. Do not repeat the title of the competition as the domain.

**Clarifications:**
**Business Industry:**  Identify the business or industry area where the competition is applied.

**Technical Category:**  Identify the broad method or approach used in the competition, such as machine learning, deep learning or any other data related field.

**Model Concept:**  Identify the specific type of AI/ML/DL modeling task required to solve the competition problem, such as computer vision, natural language processing (NLP), regression, time-series forecasting, or anomaly detection. This describes the **technical modeling focus**.

**Evaluation Metric:** if mentioned or inferred

If any field is unclear from the competition description, write `Unknown` for that field.

Your output must follow this structure exactly, no extra text:
Industry: <answer or Unknown>
Technical Category: <answer or Unknown>
Model Concept: <answer or Unknown>
Evaluation Metric: <answer or Unknown>"""

user_prompt = """
Classify this competition:
Title: Statoil/C-CORE Iceberg Classifier Challenge
Subtitle: Ship or iceberg, can you decide from space?
Overview: # Description\n\n<p>\n<a href="https://imgur.com/q7uAjTM"><img src="https://i.imgur.com/q7uAjTM.jpg" title="source: imgur.com"  width="300" style="float: right;"  /></a>\n</p>\n\n<p>\nDrifting icebergs present threats to navigation and activities in areas such as offshore of the East Coast of Canada.\n</p>\n\n<p>Currently, many institutions and companies use aerial reconnaissance and shore-based support to monitor environmental conditions and assess risks from icebergs.  However, in remote areas with particularly harsh weather, these methods are not feasible, and the only viable monitoring option is via satellite.</p>\n\n<p><a href="https://www.statoil.com/">Statoil</a>, an international energy company operating worldwide, has worked closely with companies like <a href="https://www.c-core.ca/">C-CORE</a>. C-CORE have been using satellite data for over 30 years and have built a computer vision based surveillance system. To keep operations safe and efficient, Statoil is interested in getting a fresh new perspective on how to use machine learning to more accurately detect and discriminate against threatening icebergs as early as possible.</p>\n\n<p>In this competition, you’re challenged to build an algorithm that automatically identifies if a remotely sensed target is a ship or iceberg. Improvements made will help drive the costs down for maintaining safe working conditions.</p>\n----------------------\n# Evaluation\n\nSubmissions are evaluated on the\xa0[log loss](https://www.kaggle.com/wiki/LogLoss) between the predicted values and the ground truth.</p>\n\n# Submission File\n\nFor each id\xa0in the test set, you must predict the probability that the image contains an iceberg (a number between 0 and 1). The file should contain a header and have the following format:\n\n\n<pre>id,is_iceberg<br />809385f7,0.5<br />7535f0cd,0.4<br />3aa99a38,0.9<br />etc.</pre>\n----------------------\n# Prizes\n\n<p>See the Rules Section labeled "Prizes" for the terms on receiving prize money.</p>\n<ul>\n<li>1st place - $25,000</li>\n<li>2nd place - $15,000</li>\n<li>3rd place - $10,000</li>\n</ul>\n----------------------\n# Timeline\n\n<ul>\n<li><strong>January 16, 2018</strong>\xa0-\xa0Entry deadline. You must accept the competition rules before this date in order to compete.</li>\n<li><strong>January 16, 2018</strong>\xa0- Team Merger deadline. This is the last day participants\xa0may join or merge teams.</li>\n<li><strong>January 23, 2018</strong>\xa0- Final submission deadline.</li>\n</ul>\n<p>All deadlines are at 11:59 PM UTC on the corresponding day unless otherwise noted. The competition organizers reserve the right to update the contest timeline if they deem it necessary.</p>\n----------------------\n# Background\n\n<p><img style="float: right;" src="https://storage.googleapis.com/kaggle-media/competitions/statoil/SO6VDSfm.jpg" width="271" height="203" /></p>\n<p align="justify">\n\nThe remote sensing systems used to detect icebergs are housed on satellites over 600 kilometers above the Earth. The Sentinel-1 satellite constellation is used to monitor Land and Ocean. Orbiting 14 times a day, the satellite captures images of the Earth\'s surface at a given location, at a given instant in time. The C-Band radar operates at a frequency that "sees" through darkness, rain, cloud and even fog. Since it emits it\'s own energy source it can capture images day or night. </p>\n<p align="justify">Satellite radar works in much the same way as blips on a ship or aircraft radar. It bounces a signal off an object and records the echo, then that data is translated into an image. An object will appear as a bright spot because it reflects more radar energy than its surroundings, but strong echoes can come from anything solid - land, islands, sea ice, as well as icebergs and ships. The energy reflected back to the radar is referred to as backscatter.</p>\n\n<p><img src="https://storage.googleapis.com/kaggle-media/competitions/statoil/NM5Eg0Q.png" width="829" height="323" /></p>\n\n<p><img style="float: right;" src="https://storage.googleapis.com/kaggle-media/competitions/statoil/lhYaHT0.png" /></p>\n\n<p align="justify">When the radar detects a object, it can\'t tell an iceberg from a ship or any other solid object. The object needs to be analyzed for certain characteristics - shape, size and brightness - to find that out. The area surrounding the object, in this case ocean, can also be analyzed or modeled. Many things affect the backscatter of the ocean or background area. High winds will generate a brighter background. Conversely, low winds will generate a darker background. The Sentinel-1 satellite is a side looking radar, which means it sees the image area at an angle (incidence angle). Generally, the ocean background will be darker at a higher incidence angle. You also need to consider the radar polarization, which is how the radar transmits and receives the energy. More advanced radars like Sentinel-1, can transmit and receive in the horizontal and vertical plane. Using this, you can get what is called a dual-polarization image.</p>\n\n<p align="justify">For this contest you will see data with two channels: HH (transmit/receive horizontally) and HV (transmit horizontally and receive vertically). This can play an important role in the object characteristics, since objects tend to reflect energy differently. Easy classification examples are see below. These objects can be visually classified. But in an image with hundreds of objects, this is very time consuming. <img src="https://storage.googleapis.com/kaggle-media/competitions/statoil/8ZkRcp4.png" /><img src="https://storage.googleapis.com/kaggle-media/competitions/statoil/M8OP2F2.png" /></p>\n\n<p>Here we see challenging objects to classify. We have given you the answer, but can you automate the answer to the question .... Is it a Ship or is it an Iceberg? <img src="https://storage.googleapis.com/kaggle-media/competitions/statoil/AR4NDrK.png" /><img src="https://storage.googleapis.com/kaggle-media/competitions/statoil/nXK6Vdl.png" /

---

**Provide your answer STRICTLY in this format (no explanations, no extra text):**

Industry: <your answer>
Technical Category: <your answer>
Model Concept: <your answer>
Evaluation Metric: <your answer>
"""

final_prompt = (
    B_SYS + system_prompt + E_SYS +
    B_INST + user_prompt + E_INST +
    "<start_of_turn>model\n"
)

In [ ]:
import time
start = time.time()
response = pipe(final_prompt, max_new_tokens=300, do_sample=False)
print(response[0]['generated_text'])
end = time.time()
print(start-end)

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kaggle/meta-kaggle")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/meta-kaggle


In [4]:
import pandas as pd
# Load datasets
competitionTags = pd.read_csv('/kaggle/input/meta-kaggle/CompetitionTags.csv')
competitions = pd.read_csv('/kaggle/input/meta-kaggle/Competitions.csv')

## automate competition info extract

In [5]:
def generate_prompt(row):
    B_INST, E_INST = "<start_of_turn>user\n", "<end_of_turn>\n"
    B_SYS, E_SYS = "<start_of_turn>system\n", "<end_of_turn>\n"

    system_prompt = """You are an expert AI/ML competition analyst.
Your task is to classify machine learning competitions into precise categories.
Your answer MUST use standardized academic or industry terms.
If information is unclear, answer with 'Unknown'.
Do not guess. Do not repeat the title of the competition as the domain.

**Clarifications:**
**Business Industry:** Identify the business or industry area where the competition is applied.
**Technical Category:** Identify the broad method or approach used in the competition, such as machine learning, deep learning or any other data related field.
**Model Concept:** Identify the specific type of AI/ML/DL modeling task required to solve the competition problem, such as computer vision, natural language processing (NLP), regression, time-series forecasting, or anomaly detection. This describes the **technical modeling focus**.
**Evaluation Metric:** if mentioned or inferred

If any field is unclear from the competition description, write `Unknown` for that field.

Your output must follow this structure exactly, no extra text:
Business Industry: <answer or Unknown>
Technical Category: <answer or Unknown>
Model Concept: <answer or Unknown>
Evaluation Metric: <answer or Unknown>"""

    user_prompt = f"""
Classify this competition:
Title: {row['Title']}
Subtitle: {row['Subtitle']}
Overview: {row['Overview']}

---

**Provide your answer STRICTLY in this format (no explanations, no extra text):**

Business Industry: <your answer>
Technical Category: <your answer>
Model Concept: <your answer>
Evaluation Metric: <your answer>
"""

    final_prompt = (
        B_SYS + system_prompt + E_SYS +
        B_INST + user_prompt + E_INST +
        "<start_of_turn>model\n"
    )

    return final_prompt

In [6]:
prompts = competitions.head(20).apply(generate_prompt, axis=1).tolist()

In [8]:
def batch(iterable, batch_size):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

batches = list(batch(prompts, 10))

In [ ]:
start = time.time()
results = []
for batch_prompts in batches:
    outputs = pipe(batch_prompts, max_new_tokens=300, do_sample=False)
    texts = [out[0]['generated_text'] for out in outputs]
    results.extend(texts)
end = time.time()

In [ ]:
results[0]

In [18]:
def extract_after_model(output):
    # Find the part after <start_of_turn>model
    if "<start_of_turn>model" not in output:
        return {'Industry': 'Unknown', 'Technical Category': 'Unknown', 'Model Concept': 'Unknown', 'Evaluation Metric': 'Unknown'}

    final_block = output.split("<start_of_turn>model")[-1].strip()
    lines = final_block.splitlines()

    # Clean up empty lines and whitespace
    lines = [line.strip() for line in lines if line.strip()]

    # Expecting exactly these 4 lines
    result = {}
    for line in lines:
        if ':' in line:
            key, value = line.split(':', 1)
            result[key.strip()] = value.strip()

    # Ensure all keys are present
    fields = ['Business Industry', 'Technical Category', 'Model Concept', 'Evaluation Metric']
    return {field: result.get(field, 'Unknown') for field in fields}

In [27]:
extract_after_model(results[0])

{'Industry': '<Business Industry>',
 'Technical Category': '<Machine Learning>',
 'Model Concept': '<Model Concept>',
 'Evaluation Metric': '<Absolute Error>'}

In [ ]:
results[0]

In [9]:
from accelerate import Accelerator

accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)

In [11]:
prompts_gpu0 = prompts[:len(prompts)//2]
prompts_gpu1 = prompts[len(prompts)//2:]

model0 = model.to("cuda:0")
model1 = model.to("cuda:1")

In [15]:
# Step 5: Inference Function
import torch
def generate_on_gpu(model, prompts, device):
    results = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results.append(text)
        torch.cuda.empty_cache()
    return results

In [ ]:
# Step 6: Parallel Inference on Both GPUs (Optional for Speed)
from threading import Thread
import torch

results_gpu0, results_gpu1 = [], []

def run_gpu0():
    global results_gpu0
    results_gpu0 = generate_on_gpu(model0, prompts_gpu0, "cuda:0")

def run_gpu1():
    global results_gpu1
    results_gpu1 = generate_on_gpu(model1, prompts_gpu1, "cuda:1")

import time
start = time.time()

thread0 = Thread(target=run_gpu0)
thread1 = Thread(target=run_gpu1)

thread0.start()
thread1.start()
thread0.join()
thread1.join()

end = time.time()
print(f"Total inference time: {end - start:.2f} seconds")

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)
import time
start = time.time()
results = []
for batch_prompts in batches:
    outputs = pipe(batch_prompts, max_new_tokens=300, do_sample=False)
    texts = [out[0]['generated_text'] for out in outputs]
    results.extend(texts)
end = time.time()

In [31]:
end- start

169.19261527061462

In [32]:
170/20

8.5

In [ ]:
import time
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

start = time.time()

results = []
for prompt in prompts:  # Assuming `prompts` is your list of individual prompts
    output = pipe(prompt, max_new_tokens=50, do_sample=False)
    text = output[0]['generated_text']
    results.append(text)

end = time.time()

print(f"Total time: {end - start:.2f} seconds")

In [ ]:
from datasets import Dataset
import time
# Assuming prompts is your list of strings
data = {"text": prompts}
dataset = Dataset.from_dict(data)
start2 = time.time()
def generate_completion(example):
    output = pipe(
        example["text"],
        max_new_tokens=50,
        do_sample=False
    )
    example["generated_text"] = output[0]["generated_text"]
    return example

# Run map in batched mode, more efficient for GPUs
dataset = dataset.map(generate_completion, batched=True, batch_size=2)
end2 = time.time()

## Pipeline

In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from threading import Thread
import time

# Model Loading (Half Precision Saves Memory)
model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model0 = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda:0")
model1 = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda:1")



tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
def generate_prompt(row):
    overview = str(row['Overview'])
    if len(overview) > 5000:
        overview = overview[:5000]
    B_INST, E_INST = "<start_of_turn>user\n", "<end_of_turn>\n"
    B_SYS, E_SYS = "<start_of_turn>system\n", "<end_of_turn>\n"

    system_prompt = """You are an expert AI/ML competition analyst.
Your task is to classify machine learning competitions into precise categories.
Your answer MUST use standardized academic or industry terms.
If information is unclear, answer with 'Unknown'.
Do not guess. Do not repeat the title of the competition as the domain.

**Clarifications:**
**Business Industry:** Identify the business or industry area where the competition is applied.
**AI/ML Approach:**  this is about Machine Learning, Deep Learning, Computer Vision, NLP, Generative AI or any other relevant field. Find out the right one.
**Model Type:** Just find out whether it is classification, regression, Object detection, image genaration,Time-Series Forecasting or any other related Machine learning field. Find out the right one.
**Evaluation Metric:** if mentioned or inferred

If any field is unclear from the competition description, write `Unknown` for that field.

Your output must follow this structure exactly, no extra text:
Business Industry: <answer or Unknown>
Technical Category: <answer or Unknown>
Model type: <answer or Unknown>
Evaluation Metric: <answer or Unknown>"""

    user_prompt = f"""
Classify this competition:
Title: {row['Title']}
Subtitle: {row['Subtitle']}
Overview: {overview}

---

Your output must be a valid JSON object with the following structure:
{{
  "business_industry": "<answer or Unknown>",
  "technical_category": "<answer or Unknown>",
  "model_concept": "<answer or Unknown>",
  "evaluation_metric": "<answer or Unknown>"
}}
"""

    final_prompt = (
        B_SYS + system_prompt + E_SYS +
        B_INST + user_prompt + E_INST +
        "<start_of_turn>model\n"
    )

    return final_prompt

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kaggle/meta-kaggle")

print("Path to dataset files:", path)

import pandas as pd
# Load datasets
competitionTags = pd.read_csv('/kaggle/input/meta-kaggle/CompetitionTags.csv')
competitions = pd.read_csv('/kaggle/input/meta-kaggle/Competitions.csv')

Path to dataset files: /kaggle/input/meta-kaggle


In [16]:
prompts = competitions.apply(generate_prompt, axis=1).tolist()

In [17]:
len(prompts)

9816

In [18]:
# Split prompts
prompts_gpu0 = prompts[:len(prompts)//2]
prompts_gpu1 = prompts[len(prompts)//2:]

In [19]:
len(prompts_gpu0), len(prompts_gpu1)

(4908, 4908)

In [20]:
def generate_on_gpu(model, prompts, device):
    results = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=300,
            do_sample=False
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results.append(text)
        torch.cuda.empty_cache()
    return results

In [21]:
results_gpu0, results_gpu1 = [], []

def run_gpu0():
    global results_gpu0
    results_gpu0 = generate_on_gpu(model0, prompts_gpu0, "cuda:0")

def run_gpu1():
    global results_gpu1
    results_gpu1 = generate_on_gpu(model1, prompts_gpu1, "cuda:1")

start = time.time()

thread0 = Thread(target=run_gpu0)
thread1 = Thread(target=run_gpu1)

thread0.start()
thread1.start()
thread0.join()
thread1.join()

end = time.time()
print(f"Total inference time: {end - start:.2f} seconds")

Total inference time: 36752.66 seconds


In [22]:
final_results = results_gpu0 + results_gpu1
competitions["LLM_Output"] = final_results

In [23]:
competitions.to_csv("/kaggle/working/competitions_with_extracted_info.csv", index=False)

In [26]:
competitions.to_csv("/kaggle/working/competitions_with_extracted_info.csv", index=False)

In [25]:
len(final_results)

9816

In [27]:
import re
import json

def extract_json_from_output(output):
    # Search **last 500 characters** only (to ignore the prompt repetition)
    tail = output[-500:]

    # Find the **last JSON block**
    matches = list(re.finditer(r"\{.*?\}", tail, re.DOTALL))
    if matches:
        json_str = matches[-1].group(0)  # Take the last JSON block
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError:
            data = {}
    else:
        data = {}

    # Post-clean <answer or Unknown> to Unknown
    fields = ['business_industry', 'technical_category', 'model_concept', 'evaluation_metric']
    clean_data = {}
    for field in fields:
        value = data.get(field, 'Unknown')
        if value == "<answer or Unknown>":
            value = "Unknown"
        clean_data[field] = value
    return clean_data

In [28]:
extracted_results = competitions["LLM_Output"].apply(extract_json_from_output)
extracted_df = extracted_results.apply(pd.Series)
competitions_df = competitions.join(extracted_df)

In [32]:
competitions_df['evaluation_metric'].value_counts()

evaluation_metric
Accuracy                               1888
RMSE                                    758
Mean F1-Score                           673
Unknown                                 600
AUC                                     318
                                       ... 
MSE Score                                 1
Quality of insights                       1
Micro F1 score                            1
Recall Rate                               1
Real vs. AI Classification accuracy       1
Name: count, Length: 1189, dtype: int64

In [33]:
competitions_df.to_csv("/kaggle/working/competitions_df.csv", index=False)

In [137]:
competitions_df.iloc[2]

Id                                                                               2438
Slug                                                                     worldcup2010
Title                                             World Cup 2010 - Take on the Quants
Subtitle                            Quants at Goldman Sachs and JP Morgan have mod...
HostSegmentTitle                                                             Featured
ForumId                                                                     3094129.0
OrganizationId                                                                    NaN
EnabledDate                                                       06/03/2010 08:08:08
DeadlineDate                                                      06/11/2010 13:29:00
ProhibitNewEntrantsDeadlineDate                                                   NaN
TeamMergerDeadlineDate                                                            NaN
TeamModelDeadlineDate                                 

In [140]:
competitions_df[competitions_df['business_industry']=='Unknown']['LLM_Output'].iloc[0]

'system\nYou are an expert AI/ML competition analyst.\nYour task is to classify machine learning competitions into precise categories.\nYour answer MUST use standardized academic or industry terms.\nIf information is unclear, answer with \'Unknown\'.\nDo not guess. Do not repeat the title of the competition as the domain.\n\n**Clarifications:**\n**Business Industry:** Identify the business or industry area where the competition is applied.\n**AI/ML Approach:**  this is about Machine Learning, Deep Learning, Computer Vision, NLP, Generative AI or any other relevant field. Find out the right one.\n**Model Type:** Just find out whether it is classification, regression, Object detection, image genaration,Time-Series Forecasting or any other related Machine learning field. Find out the right one.\n**Evaluation Metric:** if mentioned or inferred\n\nIf any field is unclear from the competition description, write `Unknown` for that field.\n\nYour output must follow this structure exactly, no e